In [1]:
#Imports
%matplotlib inline
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
#Network parameters
batch_size = 64
img_height = 224
img_width = 224

In [3]:
#Process data
preprocessor = keras.applications.resnet50.preprocess_input
data_generator = ImageDataGenerator(rescale=1./255, preprocessing_function=preprocessor, validation_split=0.2)


In [4]:
data_folder = '../images/uk_us/train'
training_set = data_generator.flow_from_directory(data_folder, class_mode='binary',
                                                 target_size=(img_height,img_width),
                                                 subset="training", shuffle=True, seed=666,
                                                  batch_size=batch_size)
validation_set = data_generator.flow_from_directory(data_folder, class_mode='binary',
                                                    target_size=(img_height,img_width),
                                                    subset="validation", shuffle=True, seed=666,
                                                    batch_size=batch_size)

Found 48000 images belonging to 2 classes.
Found 12000 images belonging to 2 classes.


In [5]:
#Get pretrained model
resnet_model = tf.keras.applications.ResNet50(input_shape=(img_width, img_height, 3), include_top=False, pooling='avg', weights='imagenet')
resnet_model.trainable = False
resnet_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [11]:
inputs = keras.Input(shape=(img_height,img_width, 3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = resnet_model(inputs, training=False)
x = keras.layers.Dense(1, name='logits')(x)
outputs = keras.layers.Activation('sigmoid',name='sigmoid_out')(x)
model = keras.Model(inputs, outputs)

In [12]:
#compile and set params
opt = keras.optimizers.Adam(learning_rate=0.005)
loss = keras.losses.BinaryCrossentropy(from_logits=True)
checkpoint = ModelCheckpoint('./model_checkpoints/uk_us/seq_location_classifier', save_weights_only=True, monitor="val_loss", save_best_only=True)
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='bin_accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
    keras.metrics.Accuracy(name='accuracy'),
    keras.metrics.MeanAbsoluteError(name='mae')
]

model.compile(loss=loss, optimizer=opt, metrics=METRICS)

In [13]:
 #train
history = model.fit(training_set, epochs=10, validation_data=validation_set, callbacks=[checkpoint])

Epoch 1/10
750/750 [==============================] - 6008s 8s/step - loss: 0.6296 - tp: 8473.0679 - fp: 4690.3289 - tn: 7339.4834 - fn: 3561.0346 - bin_accuracy: 0.6400 - precision: 0.6283 - recall: 0.6837 - auc: 0.6888 - accuracy: 0.0000e+00 - mae: 0.4433 - val_loss: 0.5276 - val_tp: 4476.0000 - val_fp: 1453.0000 - val_tn: 4547.0000 - val_fn: 1524.0000 - val_bin_accuracy: 0.7519 - val_precision: 0.7549 - val_recall: 0.7460 - val_auc: 0.8270 - val_accuracy: 0.0000e+00 - val_mae: 0.3790
Epoch 2/10
750/750 [==============================] - 5967s 8s/step - loss: 0.5592 - tp: 9071.4607 - fp: 3986.0919 - tn: 8044.9174 - fn: 2961.4447 - bin_accuracy: 0.7088 - precision: 0.6914 - recall: 0.7500 - auc: 0.7836 - accuracy: 0.0000e+00 - mae: 0.3924 - val_loss: 0.5104 - val_tp: 4297.0000 - val_fp: 1171.0000 - val_tn: 4829.0000 - val_fn: 1703.0000 - val_bin_accuracy: 0.7605 - val_precision: 0.7858 - val_recall: 0.7162 - val_auc: 0.8445 - val_accuracy: 0.0000e+00 - val_mae: 0.3640
Epoch 3/10
750/7

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [14]:
test_folder = '../images/uk_us/test'
preprocessor = keras.applications.resnet50.preprocess_input
test_data_generator = ImageDataGenerator(rescale=1./255, preprocessing_function=preprocessor)
test_set = test_data_generator.flow_from_directory(test_folder, class_mode='binary',target_size=(img_height,img_width))

     |████████████████████████████████| 25.9 MB 12.2 MB/s eta 0:00:01


In [ ]:
model.load_weights('./model_checkpoints/uk_us/location_classifier')

In [ ]:
model.evaluate(test_set)

In [ ]:
import pandas as pd
training_info = pd.DataFrame(history.history)
training_info.head()

In [ ]:
training_info.to_csv('./model_checkpoints/uk_us/location_classifier.csv')